# ChurnInsight — Modelo de Churn com XGBoost (Telco) 📉🤖

Este notebook implementa um **MVP de predição de churn** (cancelamento) para empresas de **serviços recorrentes** (Telecom, Fintech, Streaming, E-commerce).

**Objetivos**
- Explorar e entender o dataset (EDA)
- Preparar dados e criar features simples
- Treinar um modelo de **classificação binária** com **XGBoost**
- Avaliar desempenho com métricas clássicas
- Calibrar threshold e faixas de risco (baixo/médio/alto)
- Fazer **tuning** (RandomizedSearch) com foco em **PR-AUC** e **Recall**
- Serializar o **pipeline completo** para uso em uma API (FastAPI/Spring Boot)

📦 Dataset: `WA_Fn-UseC_-Telco-Customer-Churn.csv`

<a id="menu"></a>
## Sumário 🧭

1. [Imports e Configuração](#sec-01)  
2. [Carregamento dos Dados](#sec-02)  
3. [EDA — Análise Exploratória](#sec-03)  
4. [Limpeza e Preparação de X e y](#sec-04)  
5. [Pré-processamento e Engenharia de Atributos](#sec-05)  
6. [Modelo e Pipeline (Baseline)](#sec-06)  
7. [Treinamento](#sec-07)  
8. [Avaliação](#sec-08)  
9. [Serialização do Pipeline (Baseline)](#sec-09)  
10. [Funções de Inferência (Produção/API)](#sec-10)  
11. [Exemplo de Previsão](#sec-11)  
12. [Calibração de Risco (Baixo/Médio/Alto)](#sec-12)  
13. [Fine-tune do Modelo (RandomizedSearch)](#sec-13)  
14. [Serialização do Artefato Tunado](#sec-14)  
15. [Inferência com Artefato Tunado](#sec-15)  
16. [Relatório Baseline vs Tuned](#sec-16)  
17. [Conclusões e narrativa para banca](#sec-17)  
18. [Exemplos de requisição (payloads)](#sec-18)  
19. [Testes via cURL](#sec-19)

<a id="sec-01"></a>
## 1. Imports e Configuração ⚙️

In [ ]:
import sys, time, os
from pathlib import Path

print("Python:", sys.executable)
print("Versão:", sys.version)

mods = [
    ("pandas", "import pandas as pd"),
    ("numpy", "import numpy as np"),
    ("seaborn", "import seaborn as sns"),
    ("matplotlib", "import matplotlib.pyplot as plt"),
    ("sklearn", "import sklearn"),
    ("joblib", "import joblib"),
    ("xgboost", "import xgboost"),
]

for name, stmt in mods:
    t0 = time.time()
    print(f"Importando {name}...", end=" ")
    exec(stmt)
    print(f"OK ({time.time()-t0:.2f}s)")

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick  # formatação de eixos em %

sns.set(style="whitegrid")

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    ConfusionMatrixDisplay,
    average_precision_score,
    precision_recall_curve,
    roc_curve,
)

from xgboost import XGBClassifier
import joblib

pd.set_option("display.max_columns", 80)

[⬆ Voltar ao Sumário](#menu)

<a id="sec-02"></a>
## 2. Carregamento dos Dados 📥

In [ ]:
from pathlib import Path

def resolver_caminho_dataset(nome="WA_Fn-UseC_-Telco-Customer-Churn.csv"):
    # Caminhos comuns do projeto (notebooks/ -> ../data/raw)
    cwd = Path.cwd()
    candidatos = [
        cwd / "data" / "raw" / nome,
        cwd.parent / "data" / "raw" / nome,
        cwd.parent.parent / "data" / "raw" / nome,
    ]
    for p in candidatos:
        if p.exists():
            return p
    raise FileNotFoundError(
        "Dataset não encontrado. Garanta que exista em data/raw/ e rode o notebook a partir do projeto. "
        f"Caminhos testados: {[str(c) for c in candidatos]}"
    )

dataset_path = resolver_caminho_dataset()
print("📦 Dataset:", dataset_path)

df = pd.read_csv(dataset_path)
print("Formato do dataset:", df.shape)
df.head()

[⬆ Voltar ao Sumário](#menu)

<a id="sec-03"></a>
## 3. EDA — Análise Exploratória 🔎

In [ ]:
print("=== Info do DataFrame ===")
df.info()

print("\n=== Distribuição de Churn (absoluta) ===")
print(df["Churn"].value_counts())

print("\n=== Distribuição de Churn (proporcional) ===")
print(df["Churn"].value_counts(normalize=True))

### 3.1 Preparação leve para EDA visual

In [ ]:
df_eda = df.copy()

# TotalCharges vem como string com espaços no Telco; converter para numérico para gráficos
df_eda["TotalCharges"] = pd.to_numeric(df_eda["TotalCharges"].replace(" ", np.nan), errors="coerce")

# Target auxiliar binária para taxas (%)
df_eda["Churn_bin"] = (df_eda["Churn"] == "Yes").astype(int)

num_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
cat_cols_recomendadas = [
    "Contract", "PaymentMethod", "InternetService",
    "OnlineSecurity", "TechSupport", "PaperlessBilling",
    "SeniorCitizen", "Partner", "Dependents"
]

display(df_eda[num_cols].describe())

### 3.2 Missing Values por Coluna

In [ ]:
miss = df_eda.isna().mean().sort_values(ascending=False)
miss = miss[miss > 0]

plt.figure(figsize=(10, 4))
ax = sns.barplot(x=miss.index, y=miss.values)
plt.title("Percentual de valores ausentes por coluna")
plt.ylabel("% ausente")
plt.xlabel("Colunas")
plt.xticks(rotation=45, ha="right")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.tight_layout()
plt.show()

### 3.3 Taxa de churn global (indicador)

In [ ]:
taxa_churn_global = df_eda["Churn_bin"].mean()
CHURN_BASELINE = taxa_churn_global  # usado em gráficos posteriores
print(f"📌 Taxa global de churn no dataset: {taxa_churn_global:.2%}")

### 3.4 Churn por variáveis categóricas (taxa %)

In [ ]:
def plot_churn_rate_by_cat(df_in, col, top_n=None):
    tmp = df_in[[col, "Churn_bin"]].copy()
    tmp[col] = tmp[col].astype(str)

    if top_n is not None:
        top = tmp[col].value_counts().head(top_n).index
        tmp = tmp[tmp[col].isin(top)]

    rate = (tmp.groupby(col)["Churn_bin"].mean()
            .sort_values(ascending=False)
            .reset_index(name="taxa_churn"))

    plt.figure(figsize=(10, 4))
    ax = sns.barplot(data=rate, x=col, y="taxa_churn")
    ax.axhline(CHURN_BASELINE, linestyle="--", linewidth=2)
    plt.title(f"Taxa de churn por {col} (linha = taxa global)")
    plt.xlabel(col)
    plt.ylabel("Taxa de churn (%)")
    plt.xticks(rotation=45, ha="right")
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    plt.tight_layout()
    plt.show()

for c in cat_cols_recomendadas:
    if c in df_eda.columns:
        plot_churn_rate_by_cat(df_eda, c)

### 3.5 Distribuições numéricas por churn (boxplot + hist)

In [ ]:
def plot_num_by_churn(df_in, col):
    plt.figure(figsize=(10, 4))
    sns.boxplot(data=df_in, x="Churn", y=col)
    plt.title(f"{col} por Churn (Boxplot)")
    plt.xlabel("Churn")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.histplot(data=df_in, x=col, hue="Churn", bins=30, stat="density", common_norm=False)
    plt.title(f"Distribuição de {col} por Churn")
    plt.xlabel(col)
    plt.ylabel("Densidade")
    plt.tight_layout()
    plt.show()

for c in num_cols:
    if c in df_eda.columns:
        plot_num_by_churn(df_eda.dropna(subset=[c]), c)

### 3.6 Heatmap de correlação (numéricas)

In [ ]:
corr = df_eda[num_cols + ["Churn_bin"]].corr(numeric_only=True)

plt.figure(figsize=(6, 4))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.title("Correlação (numéricas + Churn)")
plt.tight_layout()
plt.show()

### 3.7 Churn por faixas de tenure

In [ ]:
bins_tenure = [0, 3, 6, 12, 24, 36, 48, 60, 72]
df_t = df_eda.dropna(subset=["tenure"]).copy()
df_t["tenure_bin"] = pd.cut(df_t["tenure"], bins=bins_tenure, include_lowest=True)

rate_tenure = (
    df_t.groupby("tenure_bin", observed=True)["Churn_bin"]
    .mean()
    .reset_index(name="taxa_churn")
)

rate_tenure["tenure_bin_str"] = rate_tenure["tenure_bin"].astype(str)
rate_tenure = rate_tenure.sort_values("tenure_bin")

plt.figure(figsize=(10, 4))
ax = sns.lineplot(data=rate_tenure, x="tenure_bin_str", y="taxa_churn", marker="o")
ax.axhline(CHURN_BASELINE, linestyle="--", linewidth=2)
plt.title("Taxa de churn por faixa de tenure (linha = taxa global)")
plt.xlabel("Faixa de tenure (meses)")
plt.ylabel("Taxa de churn (%)")
plt.xticks(rotation=45, ha="right")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.tight_layout()
plt.show()

[⬆ Voltar ao Sumário](#menu)

<a id="sec-04"></a>
## 4. Limpeza e Preparação de X e y 🧹

In [ ]:
# Converter TotalCharges para numérico (há valores em branco representados como espaço)
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].replace(" ", np.nan), errors="coerce")

# Preencher valores ausentes em TotalCharges com a mediana
df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].median())

# Remover qualquer linha eventualmente sem target
df = df.dropna(subset=["Churn"])

# Target binário: 0 = No, 1 = Yes
y = df["Churn"].map({"No": 0, "Yes": 1})

# Features: removemos customerID e Churn
X = df.drop(columns=["customerID", "Churn"])

print("✅ Dados prontos.")
print("X shape:", X.shape, "| y mean (taxa churn):", round(y.mean(), 4))
X.head()

[⬆ Voltar ao Sumário](#menu)

<a id="sec-05"></a>
## 5. Pré-processamento e Engenharia de Atributos 🧩

In [ ]:
numeric_features = ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]
categorical_features = [c for c in X.columns if c not in numeric_features]

print("Colunas numéricas:", numeric_features)
print("Colunas categóricas (qtd):", len(categorical_features))

preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_features),
    ]
)

preprocess

[⬆ Voltar ao Sumário](#menu)

<a id="sec-06"></a>
## 6. Modelo e Pipeline (Baseline) 🧠

In [ ]:
xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
    n_jobs=-1,
)

clf = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", xgb_model),
    ]
)

clf

[⬆ Voltar ao Sumário](#menu)

<a id="sec-07"></a>
## 7. Treinamento 🏋️

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

print("Tamanho treino:", X_train.shape)
print("Tamanho teste :", X_test.shape)

clf.fit(X_train, y_train)
print("✅ Treinamento concluído.")

[⬆ Voltar ao Sumário](#menu)

<a id="sec-08"></a>
## 8. Avaliação 📊

In [ ]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test, y_proba)

print("=== Métricas no conjunto de teste (Baseline @ thr=0.5) ===")
print(f"Acurácia : {accuracy:.4f}")
print(f"Precisão : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC AUC  : {roc_auc:.4f}")

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, zero_division=0))

disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title("Matriz de confusão - XGBoost (Baseline)")
plt.tight_layout()
plt.show()

[⬆ Voltar ao Sumário](#menu)

<a id="sec-09"></a>
## 9. Serialização do Pipeline (Baseline) 💾

In [ ]:
# Salvar pipeline completo (pré-processamento + modelo)
cwd = Path.cwd()
model_dir = (cwd.parent / "model") if (cwd.name.lower() == "notebooks") else (cwd / "model")
model_dir.mkdir(parents=True, exist_ok=True)

nome_arquivo_modelo = model_dir / "churn_xgboost_pipeline.joblib"
joblib.dump(clf, nome_arquivo_modelo)
print(f"✅ Pipeline salvo em: {nome_arquivo_modelo}")

[⬆ Voltar ao Sumário](#menu)

<a id="sec-10"></a>
## 10. Funções de Inferência (Produção/API) 🧪

In [ ]:
def carregar_modelo(caminho_modelo: str = None):
    # Carrega o pipeline de churn treinado (pré-processamento + XGBoost).
    if caminho_modelo is None:
        caminho_modelo = str(nome_arquivo_modelo)
    return joblib.load(caminho_modelo)

def prever_cliente(dados_cliente: dict, modelo=None, threshold: float = 0.5):
    # Recebe dict com os dados do cliente e retorna previsão e probabilidade.
    if modelo is None:
        modelo = carregar_modelo()

    X_novo = pd.DataFrame([dados_cliente])

    colunas_esperadas = list(X.columns)
    faltando = set(colunas_esperadas) - set(X_novo.columns)
    sobrando = set(X_novo.columns) - set(colunas_esperadas)

    if faltando:
        raise ValueError(f"❌ Faltam colunas na entrada: {sorted(list(faltando))}")
    if sobrando:
        raise ValueError(f"❌ Existem colunas não reconhecidas: {sorted(list(sobrando))}")

    X_novo = X_novo[colunas_esperadas]

    prob = float(modelo.predict_proba(X_novo)[:, 1][0])
    pred = int(prob >= threshold)

    return {
        "previsao": "Vai cancelar" if pred == 1 else "Vai continuar",
        "probabilidade": prob,
        "threshold_usado": float(threshold),
    }

[⬆ Voltar ao Sumário](#menu)

<a id="sec-11"></a>
## 11. Exemplo de Previsão 🧾

In [ ]:
exemplo_cliente = {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 12,
    "PhoneService": "Yes",
    "MultipleLines": "No",
    "InternetService": "Fiber optic",
    "OnlineSecurity": "No",
    "OnlineBackup": "Yes",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "Yes",
    "StreamingMovies": "No",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 70.35,
    "TotalCharges": 151.65,
}

modelo_carregado = carregar_modelo()
resultado = prever_cliente(exemplo_cliente, modelo=modelo_carregado, threshold=0.5)
resultado

[⬆ Voltar ao Sumário](#menu)

<a id="sec-12"></a>
## 12. Calibração de Risco (Baixo/Médio/Alto) 🚦

In [ ]:
df_eval = pd.DataFrame({
    "y_true": y_test.values,
    "prob_churn": y_proba
})

df_eval.head()

### 12.1 Distribuição das probabilidades

In [ ]:
bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = ["0–0.2", "0.2–0.4", "0.4–0.6", "0.6–0.8", "0.8–1.0"]

df_eval["prob_bin"] = pd.cut(
    df_eval["prob_churn"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

dist_prob = df_eval["prob_bin"].value_counts().sort_index()
print("=== Distribuição de clientes por faixa de probabilidade ===")
print(dist_prob)

### 12.2 Churn real por faixa

In [ ]:
stats_bins = (
    df_eval
    .groupby("prob_bin", observed=True)
    .agg(
        total_clientes=("y_true", "count"),
        churns_reais=("y_true", "sum"),
    )
)

stats_bins["taxa_churn_real"] = stats_bins["churns_reais"] / stats_bins["total_clientes"]

print("=== Churn real por faixa de probabilidade prevista ===")
display(stats_bins)

taxa_churn_global_teste = df_eval["y_true"].mean()
print(f"\n📌 Taxa global de churn no teste: {taxa_churn_global_teste:.4f}")

### 12.3 Teste de cutoffs para risco (Baixo/Médio/Alto)

In [ ]:
def classificar_risco(prob, thr_medio, thr_alto):
    if prob >= thr_alto:
        return "alto"
    elif prob >= thr_medio:
        return "medio"
    else:
        return "baixo"

thresholds_medio = [0.3, 0.4, 0.5]
thresholds_alto  = [0.6, 0.7, 0.8]

resultados = []

for thr_m in thresholds_medio:
    for thr_a in thresholds_alto:
        if thr_a <= thr_m:
            continue

        df_tmp = df_eval.copy()
        df_tmp["risco"] = df_tmp["prob_churn"].apply(lambda p: classificar_risco(p, thr_m, thr_a))

        resumo = (
            df_tmp
            .groupby("risco")
            .agg(
                total_clientes=("y_true", "count"),
                churns_reais=("y_true", "sum")
            )
        )

        resumo["taxa_churn_real"] = resumo["churns_reais"] / resumo["total_clientes"]

        resultados.append({
            "thr_medio": thr_m,
            "thr_alto": thr_a,
            "total_alto": resumo.loc["alto", "total_clientes"] if "alto" in resumo.index else 0,
            "taxa_alto": resumo.loc["alto", "taxa_churn_real"] if "alto" in resumo.index else float("nan"),
            "total_medio": resumo.loc["medio", "total_clientes"] if "medio" in resumo.index else 0,
            "taxa_medio": resumo.loc["medio", "taxa_churn_real"] if "medio" in resumo.index else float("nan"),
            "total_baixo": resumo.loc["baixo", "total_clientes"] if "baixo" in resumo.index else 0,
            "taxa_baixo": resumo.loc["baixo", "taxa_churn_real"] if "baixo" in resumo.index else float("nan"),
        })

df_thresholds = pd.DataFrame(resultados)
df_thresholds.sort_values("taxa_alto", ascending=False).head(10)

[⬆ Voltar ao Sumário](#menu)

<a id="sec-13"></a>
## 13. Fine-tune do Modelo (RandomizedSearch) 🎛️⚡

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

X_train_ft, X_val, y_train_ft, y_val = train_test_split(
    X_train, y_train,
    test_size=0.25,
    random_state=42,
    stratify=y_train
)

print("📦 Shapes:")
print(" - Train (FT):", X_train_ft.shape)
print(" - Val       :", X_val.shape)
print(" - Test      :", X_test.shape)

n_pos = int((y_train_ft == 1).sum())
n_neg = int((y_train_ft == 0).sum())
scale_pos_weight = n_neg / max(n_pos, 1)

print(f"⚖️ Desbalanceamento no treino (FT): pos={n_pos}, neg={n_neg}")
print(f"✅ scale_pos_weight = {scale_pos_weight:.4f}")

### 13.1 Configuração do tuning

In [ ]:
N_ITER = 25
CV_SPLITS = 5
N_JOBS = -1  # se estiver no Free Tier, considere 1 ou 2

xgb_model_tune = XGBClassifier(
    objective="binary:logistic",
    eval_metric="aucpr",
    random_state=42,
    n_jobs=N_JOBS,
    tree_method="hist",
    scale_pos_weight=scale_pos_weight
)

clf_tune = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", xgb_model_tune),
    ]
)

cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=42)

param_distributions = {
    "model__n_estimators": [200, 300, 400, 600, 800],
    "model__max_depth": [2, 3, 4, 5, 6],
    "model__learning_rate": [0.02, 0.05, 0.08, 0.1, 0.15, 0.2],
    "model__subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "model__colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
    "model__min_child_weight": [1, 2, 3, 5, 7, 10],
    "model__gamma": [0, 0.5, 1, 2, 5],
    "model__reg_alpha": [0, 0.01, 0.05, 0.1, 0.5, 1.0],
    "model__reg_lambda": [0.5, 1.0, 2.0, 3.0, 5.0],
    "model__max_delta_step": [0, 1, 5],
    "model__scale_pos_weight": [scale_pos_weight * 0.75, scale_pos_weight, scale_pos_weight * 1.25],
}

search = RandomizedSearchCV(
    estimator=clf_tune,
    param_distributions=param_distributions,
    n_iter=N_ITER,
    scoring="average_precision",
    cv=cv,
    verbose=1,
    n_jobs=N_JOBS,
    random_state=42,
    refit=True
)

print("🚀 Iniciando RandomizedSearchCV...")
search.fit(X_train_ft, y_train_ft)

print("\n🏁 Tuning finalizado.")
print("⭐ Melhor PR-AUC (CV):", round(search.best_score_, 6))
print("🧩 Melhores parâmetros:", search.best_params_)

best_model = search.best_estimator_

### 13.2 Escolha de Threshold (Validação)

In [ ]:
def escolher_threshold_max_f1(y_true, proba, grid=np.linspace(0.05, 0.95, 91)):
    melhor_thr, melhor_f1 = 0.5, -1
    for thr in grid:
        pred = (proba >= thr).astype(int)
        f1 = f1_score(y_true, pred, zero_division=0)
        if f1 > melhor_f1:
            melhor_f1 = f1
            melhor_thr = float(thr)
    return melhor_thr, melhor_f1

def escolher_threshold_precision_min(y_true, proba, precision_min=0.50, grid=np.linspace(0.05, 0.95, 91)):
    melhor_thr, melhor_recall = None, -1
    for thr in grid:
        pred = (proba >= thr).astype(int)
        p = precision_score(y_true, pred, zero_division=0)
        r = recall_score(y_true, pred, zero_division=0)
        if p >= precision_min and r > melhor_recall:
            melhor_recall = r
            melhor_thr = float(thr)
    return melhor_thr, melhor_recall

val_proba = best_model.predict_proba(X_val)[:, 1]

thr_f1, val_f1 = escolher_threshold_max_f1(y_val, val_proba)
thr_prec, val_rec = escolher_threshold_precision_min(y_val, val_proba, precision_min=0.50)

print(f"🎯 Threshold (max F1) no VAL: {thr_f1:.2f} | F1={val_f1:.4f}")
if thr_prec is not None:
    print(f"📌 Threshold (Precision>=0.50) no VAL: {thr_prec:.2f} | Recall={val_rec:.4f}")
else:
    print("⚠️ Não foi possível atingir Precision>=0.50 com a grade de thresholds.")

threshold_escolhido = thr_f1
print(f"✅ Threshold escolhido para produção/API: {threshold_escolhido:.2f}")

### 13.3 Avaliação Final no Teste (Tuned + Threshold do VAL)

In [ ]:
test_proba_tuned = best_model.predict_proba(X_test)[:, 1]
test_pred_tuned = (test_proba_tuned >= threshold_escolhido).astype(int)

print("=== MÉTRICAS NO TESTE (Tuned + Threshold do VAL) ===")
print("Threshold:", round(threshold_escolhido, 4))
print("Accuracy :", round(accuracy_score(y_test, test_pred_tuned), 6))
print("Precision:", round(precision_score(y_test, test_pred_tuned, zero_division=0), 6))
print("Recall   :", round(recall_score(y_test, test_pred_tuned, zero_division=0), 6))
print("F1       :", round(f1_score(y_test, test_pred_tuned, zero_division=0), 6))
print("ROC-AUC  :", round(roc_auc_score(y_test, test_proba_tuned), 6))
print("PR-AUC   :", round(average_precision_score(y_test, test_proba_tuned), 6))

print("\n=== Classification Report (Tuned) ===")
print(classification_report(y_test, test_pred_tuned, zero_division=0))

disp = ConfusionMatrixDisplay.from_predictions(y_test, test_pred_tuned)
plt.title("Matriz de confusão - XGBoost Tunado (Churn)")
plt.tight_layout()
plt.show()

[⬆ Voltar ao Sumário](#menu)

<a id="sec-14"></a>
## 14. Serialização do Artefato Tunado 💾📦

In [ ]:
artefato_tuned = {
    "model": best_model,
    "threshold": float(threshold_escolhido),
    "features": list(X.columns),
    "numeric_features": numeric_features,
    "categorical_features": categorical_features,
    "best_params": search.best_params_,
    "best_cv_pr_auc": float(search.best_score_),
}

out_path_tuned = model_dir / "churn_xgboost_pipeline_tuned.joblib"
joblib.dump(artefato_tuned, out_path_tuned)

print(f"✅ Artefato tunado salvo em: {out_path_tuned}")

[⬆ Voltar ao Sumário](#menu)

<a id="sec-15"></a>
## 15. Inferência com Artefato Tunado 🧩🧪

In [ ]:
def carregar_artefato(caminho: str = None) -> dict:
    if caminho is None:
        caminho = str(out_path_tuned)
    artefato = joblib.load(caminho)

    obrigatorias = {"model", "threshold", "features"}
    faltando = obrigatorias - set(artefato.keys())
    if faltando:
        raise ValueError(f"❌ Artefato inválido. Faltam chaves: {faltando}")

    thr = float(artefato["threshold"])
    if not (0.0 <= thr <= 1.0):
        raise ValueError("❌ Artefato inválido: 'threshold' deve estar entre 0 e 1.")

    feats = artefato["features"]
    if not isinstance(feats, (list, tuple)) or len(feats) == 0:
        raise ValueError("❌ Artefato inválido: 'features' deve ser uma lista não vazia.")

    return artefato

def prever_cliente_com_artefato(dados_cliente: dict, artefato: dict = None) -> dict:
    if artefato is None:
        artefato = carregar_artefato()

    modelo = artefato["model"]
    threshold = float(artefato["threshold"])
    features = list(artefato["features"])

    X_novo = pd.DataFrame([dados_cliente])

    faltando = set(features) - set(X_novo.columns)
    sobrando = set(X_novo.columns) - set(features)

    if faltando:
        raise ValueError(f"❌ Entrada inválida. Faltam campos: {sorted(list(faltando))}")
    if sobrando:
        raise ValueError(f"❌ Entrada inválida. Campos não reconhecidos: {sorted(list(sobrando))}")

    X_novo = X_novo[features]

    # Coerção de numéricos (tolerância a strings numéricas vindas da API)
    numeric_feats = artefato.get("numeric_features", [])
    for col in numeric_feats:
        if col in X_novo.columns:
            X_novo[col] = pd.to_numeric(X_novo[col], errors="coerce")

    cols_nan = [c for c in numeric_feats if c in X_novo.columns and X_novo[c].isna().any()]
    if cols_nan:
        raise ValueError(f"❌ Entrada inválida. Numéricos não convertidos corretamente: {cols_nan}")

    prob = float(modelo.predict_proba(X_novo)[:, 1][0])
    pred = int(prob >= threshold)

    return {
        "previsao": "Vai cancelar" if pred == 1 else "Vai continuar",
        "probabilidade": prob,
        "threshold_usado": threshold,
    }

artefato = carregar_artefato()
resultado_tuned = prever_cliente_com_artefato(exemplo_cliente, artefato)
print("🧾 Resultado (Tuned):", resultado_tuned)

[⬆ Voltar ao Sumário](#menu)

<a id="sec-16"></a>
## 16. Relatório Baseline vs Tuned 📊🧪

### Por que o modelo tunado pode ter acurácia menor e ainda ser melhor?

Em churn, a classe **positiva** (churn=1) costuma ser minoria. Acurácia pode ficar “bonita” mesmo quando o modelo erra churn,
porque acertar “não churn” é numericamente mais fácil.

Por isso, em um MVP de churn, a banca costuma olhar com bons olhos:
- **Recall** (capturar mais churners para agir antes do cancelamento)
- **PR-AUC** (melhor leitura quando há desbalanceamento)
- **F1** (equilíbrio entre precision/recall)

Além disso, o **threshold (cutoff)** raramente deve ser 0,5. Ele é calibrado para reduzir churn “perdido” (FN) e adequar o volume
de clientes à capacidade do time de retenção.

In [ ]:
def avaliar_modelo(y_true, proba, threshold=0.5):
    pred = (proba >= threshold).astype(int)
    return {
        "threshold": float(threshold),
        "accuracy": float(accuracy_score(y_true, pred)),
        "precision": float(precision_score(y_true, pred, zero_division=0)),
        "recall": float(recall_score(y_true, pred, zero_division=0)),
        "f1": float(f1_score(y_true, pred, zero_division=0)),
        "roc_auc": float(roc_auc_score(y_true, proba)),
        "pr_auc": float(average_precision_score(y_true, proba)),
    }

baseline_metrics = avaliar_modelo(y_test, y_proba, threshold=0.5)
tuned_metrics = avaliar_modelo(y_test, test_proba_tuned, threshold=float(threshold_escolhido))

df_compare = pd.DataFrame(
    [
        {"modelo": "Baseline 🧠", **baseline_metrics},
        {"modelo": "Tuned 🎛️", **tuned_metrics},
    ]
)

df_show = df_compare.copy()
for col in ["accuracy", "precision", "recall", "f1", "roc_auc", "pr_auc"]:
    df_show[col] = df_show[col].map(lambda x: f"{x:.4f}")
df_show["threshold"] = df_show["threshold"].map(lambda x: f"{x:.2f}")

display(df_show)

delta = {k: tuned_metrics[k] - baseline_metrics[k] for k in ["accuracy","precision","recall","f1","roc_auc","pr_auc"]}
print("\n📌 Delta (Tuned - Baseline):")
for k, v in delta.items():
    print(f" - {k}: {v:+.4f}")

### 16.2 Curvas ROC e Precision-Recall

In [ ]:
fpr_b, tpr_b, _ = roc_curve(y_test, y_proba)
fpr_t, tpr_t, _ = roc_curve(y_test, test_proba_tuned)

plt.figure(figsize=(7,5))
plt.plot(fpr_b, tpr_b, label=f"Baseline (ROC-AUC={baseline_metrics['roc_auc']:.3f})")
plt.plot(fpr_t, tpr_t, label=f"Tuned (ROC-AUC={tuned_metrics['roc_auc']:.3f})")
plt.plot([0,1],[0,1], linestyle="--")
plt.title("Curva ROC - Baseline vs Tuned")
plt.xlabel("Falso Positivo (FPR)")
plt.ylabel("Verdadeiro Positivo (TPR)")
plt.legend()
plt.tight_layout()
plt.show()

prec_b, rec_b, _ = precision_recall_curve(y_test, y_proba)
prec_t, rec_t, _ = precision_recall_curve(y_test, test_proba_tuned)

plt.figure(figsize=(7,5))
plt.plot(rec_b, prec_b, label=f"Baseline (PR-AUC={baseline_metrics['pr_auc']:.3f})")
plt.plot(rec_t, prec_t, label=f"Tuned (PR-AUC={tuned_metrics['pr_auc']:.3f})")
plt.title("Curva Precision-Recall - Baseline vs Tuned")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.tight_layout()
plt.show()

[⬆ Voltar ao Sumário](#menu)

<a id="sec-17"></a>
## 17. Conclusões e narrativa para banca ✅

In [ ]:
def fmt_pct(x): 
    return f"{x*100:.2f}%"

def fmt_delta_pp(x):
    sign = "+" if x >= 0 else ""
    return f"{sign}{x*100:.2f} p.p."

texto = f'''
📌 Resumo Executivo (Baseline vs Tuned)

🧠 Baseline (thr=0.50)
- Precision: {fmt_pct(baseline_metrics['precision'])}
- Recall:    {fmt_pct(baseline_metrics['recall'])}
- F1:        {fmt_pct(baseline_metrics['f1'])}
- ROC-AUC:   {baseline_metrics['roc_auc']:.4f}
- PR-AUC:    {baseline_metrics['pr_auc']:.4f}

🎛️ Tuned (thr={tuned_metrics['threshold']:.2f})
- Precision: {fmt_pct(tuned_metrics['precision'])} ({fmt_delta_pp(delta['precision'])})
- Recall:    {fmt_pct(tuned_metrics['recall'])} ({fmt_delta_pp(delta['recall'])})
- F1:        {fmt_pct(tuned_metrics['f1'])} ({fmt_delta_pp(delta['f1'])})
- ROC-AUC:   {tuned_metrics['roc_auc']:.4f} ({delta['roc_auc']:+.4f})
- PR-AUC:    {tuned_metrics['pr_auc']:.4f} ({delta['pr_auc']:+.4f})

✅ Interpretação para o negócio:
- Para churn, muitas vezes é melhor aumentar Recall/PR-AUC do que maximizar acurácia.
- O threshold é calibrado para alinhar decisão com operação (capacidade de retenção) e evitar churn perdido.
'''
print(texto)

[⬆ Voltar ao Sumário](#menu)

<a id="sec-18"></a>
## 18. Exemplos de requisição (payloads) 🧪📨

Abaixo estão 3 payloads para testar o endpoint `POST /predict`:

- **Alto risco**: contrato mensal, pouco tempo de casa, cobrança alta, pagamento `Electronic check`.
- **Baixo risco**: contrato longo, pagamento automático, mais serviços de suporte/segurança.
- **Inválido**: faltando campo obrigatório e/ou tipo inválido.

In [ ]:
payload_alto_risco = {
  "gender": "Female",
  "SeniorCitizen": 1,
  "Partner": "No",
  "Dependents": "No",
  "tenure": 2,
  "PhoneService": "Yes",
  "MultipleLines": "No",
  "InternetService": "Fiber optic",
  "OnlineSecurity": "No",
  "OnlineBackup": "No",
  "DeviceProtection": "No",
  "TechSupport": "No",
  "StreamingTV": "Yes",
  "StreamingMovies": "Yes",
  "Contract": "Month-to-month",
  "PaperlessBilling": "Yes",
  "PaymentMethod": "Electronic check",
  "MonthlyCharges": 99.85,
  "TotalCharges": 199.70
}

payload_baixo_risco = {
  "gender": "Male",
  "SeniorCitizen": 0,
  "Partner": "Yes",
  "Dependents": "Yes",
  "tenure": 48,
  "PhoneService": "Yes",
  "MultipleLines": "Yes",
  "InternetService": "DSL",
  "OnlineSecurity": "Yes",
  "OnlineBackup": "Yes",
  "DeviceProtection": "Yes",
  "TechSupport": "Yes",
  "StreamingTV": "No",
  "StreamingMovies": "No",
  "Contract": "Two year",
  "PaperlessBilling": "No",
  "PaymentMethod": "Bank transfer (automatic)",
  "MonthlyCharges": 59.9,
  "TotalCharges": 2875.2
}

payload_invalido = {
  "SeniorCitizen": 0,
  "Partner": "Yes",
  "Dependents": "No",
  "tenure": "doze",
  "PhoneService": "Yes",
  "MultipleLines": "No",
  "InternetService": "Fiber optic",
  "OnlineSecurity": "No",
  "OnlineBackup": "Yes",
  "DeviceProtection": "No",
  "TechSupport": "No",
  "StreamingTV": "Yes",
  "StreamingMovies": "No",
  "Contract": "Month-to-month",
  "PaperlessBilling": "Yes",
  "PaymentMethod": "Electronic check",
  "MonthlyCharges": 70.35,
  "TotalCharges": 151.65
}

payload_alto_risco, payload_baixo_risco, payload_invalido

[⬆ Voltar ao Sumário](#menu)

<a id="sec-19"></a>
## 19. Testes via cURL (FastAPI) 🧰🧪

Exemplo para FastAPI rodando em `http://localhost:8000` (endpoint `POST /predict`).

No Windows PowerShell, prefira `curl.exe` para evitar alias do `Invoke-WebRequest`.

In [ ]:
curl_alto = r'''
curl -X POST "http://localhost:8000/predict" ^
  -H "Content-Type: application/json" ^
  -d "{\"gender\":\"Female\",\"SeniorCitizen\":0,\"Partner\":\"Yes\",\"Dependents\":\"No\",\"tenure\":1,\"PhoneService\":\"Yes\",\"MultipleLines\":\"No\",\"InternetService\":\"Fiber optic\",\"OnlineSecurity\":\"No\",\"OnlineBackup\":\"Yes\",\"DeviceProtection\":\"No\",\"TechSupport\":\"No\",\"StreamingTV\":\"Yes\",\"StreamingMovies\":\"No\",\"Contract\":\"Month-to-month\",\"PaperlessBilling\":\"Yes\",\"PaymentMethod\":\"Electronic check\",\"MonthlyCharges\":99,\"TotalCharges\":99}"
'''
print(curl_alto)